In [1]:
import pandas as pd
import time
import lxml
import unicodedata
import numpy as np
import math

In [80]:
url_list = ['https://www.nikkansports.com/baseball/professional/score/2020/cl2020092216.html',
'https://www.nikkansports.com/baseball/professional/score/2020/cl2020092312.html',
'https://www.nikkansports.com/baseball/professional/score/2020/cl2020092510.html',
'https://www.nikkansports.com/baseball/professional/score/2020/cl2020092612.html',
'https://www.nikkansports.com/baseball/professional/score/2020/cl2020092712.html'
]
datetime_list = ["2020-09-22", "2020-09-23", "2020-09-25", "2020-09-26", "2020-09-27"]
list_num = [0, 0, 2, 2, 2]

In [ ]:
url_list = [
'https://www.nikkansports.com/baseball/professional/score/2020/cl2020061911.html'
'https://www.nikkansports.com/baseball/professional/score/2020/cl2020062014.html'
'https://www.nikkansports.com/baseball/professional/score/2020/cl2020062114.html'
'https://www.nikkansports.com/baseball/professional/score/2020/cl2020062312.html'
'https://www.nikkansports.com/baseball/professional/score/2020/cl2020062411.html'
'https://www.nikkansports.com/baseball/professional/score/2020/cl2020062509.html'
'https://www.nikkansports.com/baseball/professional/score/2020/cl2020062612.html'
'https://www.nikkansports.com/baseball/professional/score/2020/cl2020062715.html'
'https://www.nikkansports.com/baseball/professional/score/2020/cl2020062815.html'
]
list_num = [
    0, 0, 0, 0, 0, 0, 0, 0, 0, 
    ]

In [3]:
def get_batter_data(url, order):
    data = pd.read_html(url)
    time.sleep(2)
    batter = data[order][(data[order]["守備"] != "投") & (data[order]["守備"] != "(投)")].reset_index(drop = True)[:-1]
    return batter

In [4]:
def columns_zen2han(data):
    # 全角 -> 半角に変換する辞書
    ZEN = "".join(chr(0xff01 + i) for i in range(94))
    HAN = "".join(chr(0x21 + i) for i in range(94))
    ZEN2HAN = str.maketrans(ZEN, HAN)
    new_columns = []
    for col in data.columns:
        new_col = ''
        for num_char in range(len(col)):
            if unicodedata.east_asian_width(col[num_char]) == 'F':
                new_char = col[num_char].translate(ZEN2HAN)
                new_col += new_char
            else:
                new_col += col[num_char]
        new_columns.append(new_col)
    data.columns = new_columns
    return data

In [5]:
def is_nan(x):
    return x != x

In [6]:
def contents_zen2han(data):
    # 全角 -> 半角に変換する辞書
    ZEN = "".join(chr(0xff01 + i) for i in range(94))
    HAN = "".join(chr(0x21 + i) for i in range(94))
    ZEN2HAN = str.maketrans(ZEN, HAN)

    cols = ["打数", "得点", "安打", "打点", "HR", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    for col in cols:
        new_contents = []
        for content in data[col]:
            output = ''
            if is_nan(content):
                output = content
            elif content == "……":
                new_char = np.nan
                output = new_char
            else:
                for char_num in range(len(content)):
                    if unicodedata.east_asian_width(content[char_num]) == 'F':
                        new_char = content[char_num].translate(ZEN2HAN)
                        output += new_char
                    else:
                        output += content[char_num]
            new_contents.append(output)
        data[col] = new_contents
    return data

In [88]:
score_list = []
desired = ["選手名", "打数", "得点", "安打", "打点"]
for i in range(len(url_list)):
    print(i)
    batter = get_batter_data(url_list[i], list_num[i])
    batter = columns_zen2han(batter)
    batter = contents_zen2han(batter)
    batter = batter.loc[:, desired]
    for j in desired[1:]:
        batter[j] = batter[j].astype(int)
    batter["日付"] = pd.to_datetime([datetime_list[i] for k in range(len(batter))])
    score_list.append(batter)

0
1
2
3
4


In [104]:
score_list[0].head()

,選手名,打数,得点,安打,打点,日付
0,田中広,4,0,1,0,2020-09-22
1,菊池涼,4,0,1,0,2020-09-22
2,坂倉,4,0,1,0,2020-09-22
3,曽根,0,1,0,0,2020-09-22
4,鈴木誠,4,1,1,0,2020-09-22


In [111]:
test = pd.concat([score_list[i] for i in range(5)], ignore_index=True)

In [112]:
test2 = test.groupby(["日付","選手名"]).sum()

In [113]:
test2

打数  得点  安打  打点
日付         選手名                
2020-09-22 ピレラ   1   0   0   0
           上本    0   0   0   0
           坂倉    4   0   1   0
           堂林    4   0   0   0
           大盛    3   0   1   0
           曽根    0   1   0   0
           松山    4   1   2   1
           田中広   4   0   1   0
           磯村    0   0   0   0
           菊池涼   4   0   1   0
           鈴木誠   4   1   1   0
           長野    4   1   1   3
2020-09-23 ピレラ   4   2   1   2
           メヒア   1   0   0   0
           上本    0   0   0   0
           坂倉    1   0   0   0
           堂林    2   0   1   1
           松山    2   1   1   0
           正随    1   0   0   0
           田中広   4   0   1   0
           磯村    3   0   0   0
           菊池涼   4   0   0   0
           鈴木誠   4   0   0   0
           長野    4   0   1   0
2020-09-25 ピレラ   3   0   2   1
           三好    0   0   0   0
           坂倉    4   0   0   0
           堂林    4   0   0   0
           大盛    3   0   1   1
           松山    4   1   2   1
           田中広   4   1   1   2
           菊池涼   4   0   0   0
           野間    1   0   0   0
           鈴木誠   4   2   3   0
2020-09-26 ピレラ   1   0   0   0
           メヒア   1   0   0   0
           坂倉    3   0   0   0
           堂林    4   0   1   0
           松山    3   0   1   0
           田中広   4   1   1   1
           菊池涼   4   0   1   0
           野間    3   1   1   0
           鈴木誠   4   0   1   1
           長野    4   0   0   0
2020-09-27 ピレラ   4   0   0   0
           メヒア   3   0   0   0
           坂倉    1   0   0   0
           大盛    1   0   0   0
           松山    3   1   1   1
           田中広   4   0   0   0
           磯村    2   0   0   0
           菊池涼   3   0   1   0
           野間    3   0   0   0
           鈴木誠   3   0   1   0
           長野    1   0   0   0

In [114]:
test2.loc['2020-09-22':'2020-09-27'].groupby("選手名").sum()

,打数,得点,安打,打点
選手名,,,,
ピレラ,13,2,3,3
メヒア,5,0,0,0
三好,0,0,0,0
上本,0,0,0,0
坂倉,13,0,1,0
堂林,14,0,2,1
大盛,7,0,2,1
曽根,0,1,0,0
松山,16,4,7,3


In [34]:
# 打撃成績を抽出
scores = batter.iloc[:, 9:]
scores.index = batter["選手名"]
scores = scores.T

In [35]:
scores

選手名,田中広,菊池涼,坂倉,曽根,鈴木誠,松山,上本,長野,堂林,大盛,ピレラ,磯村
1,三振,三ゴ,遊飛,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,右飛,遊ゴ,NaN,右飛,NaN,NaN,NaN,NaN
3,三振,NaN,NaN,NaN,NaN,NaN,NaN,NaN,遊ゴ,左2,NaN,NaN
4,NaN,左安,二併,NaN,中安,左安,NaN,左本,二ゴ,NaN,NaN,NaN
5,一ゴ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中飛,NaN,NaN
6,NaN,二ゴ,三振,NaN,左飛,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,遊直,NaN,三振,中飛,NaN,NaN,NaN
8,左安,三振,NaN,NaN,NaN,NaN,NaN,NaN,NaN,三振,中飛,NaN
9,NaN,NaN,中安,NaN,二飛,左安,NaN,二直,中飛,NaN,NaN,NaN


In [46]:
scores_dic = {}
for i in batter["選手名"]:
    scores_dic[i] = scores[i].dropna(axis=0).values

In [57]:
scores_dic['test'] = ["a", "b", "c", "d", "e", "f"]

In [59]:
scores_dic

{'田中広': array(['三振', '三振', '一ゴ', '左安'], dtype=object),
 '菊池涼': array(['三ゴ', '左安', '二ゴ', '三振'], dtype=object),
 '坂倉': array(['遊飛', '二併', '三振', '中安'], dtype=object),
 '曽根': array([], dtype=object),
 '鈴木誠': array(['右飛', '中安', '左飛', '二飛'], dtype=object),
 '松山': array(['遊ゴ', '左安', '遊直', '左安'], dtype=object),
 '上本': array([], dtype=object),
 '長野': array(['右飛', '左本', '三振', '二直'], dtype=object),
 '堂林': array(['遊ゴ', '二ゴ', '中飛', '中飛'], dtype=object),
 '大盛': array(['左2', '中飛', '三振'], dtype=object),
 'ピレラ': array(['中飛'], dtype=object),
 '磯村': array([], dtype=object),
 'test': ['a', 'b', 'c', 'd', 'e', 'f']}